In [ ]:
import os
import cv2
import pytesseract
from gtts import gTTS
import pygame
import time

In [ ]:
# # Obtener una lista de todas las cámaras disponibles
# available_cameras = []
# for i in range(10):  # Intenta hasta 10 cámaras (puedes ajustar este valor según tus necesidades)
#     cap = cv2.VideoCapture(i)
#     if cap.isOpened():
#         available_cameras.append(i)
#         cap.release()

# # Imprime las cámaras disponibles
# print("Cámaras disponibles:", available_cameras)

In [ ]:
# # Configura TESSDATA_PREFIX
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Ruta a Tesseract en Windows
# pytesseract.pytesseract.tessdata_dir_config = '--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'

# # Especifica el idioma que deseas utilizar en el OCR
idioma_ocr = 'spa'  # Cambia 'spa' por el código de idioma que necesitas (por ejemplo, 'en' para inglés)
idioma_tts = 'es'

# Elige la cámara que deseas usar
camera_index = 1  # Cambia esto al índice de la cámara que desees utilizar (por ejemplo, 0 para la primera cámara)

In [ ]:
# Inicializa pygame
pygame.init()

# Inicializa la cámara (puedes elegir la cámara al igual que en el ejemplo anterior)

def capture_text_from_image():
    cap = cv2.VideoCapture(camera_index)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("OCR - Stream", frame)

        # Comprueba si se ha presionado la tecla 'P' para capturar una imagen
        key = cv2.waitKey(1)
        if key == ord('p') or key == ord('P'):
            # Realiza mejoras en la imagen
            alpha = 1.5  # Ajusta este valor según tus necesidades para el contraste
            beta = 0
            frame_contrast = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

            gray_frame = cv2.cvtColor(frame_contrast, cv2.COLOR_BGR2GRAY)

            _, binary_frame = cv2.threshold(gray_frame, 128, 255, cv2.THRESH_BINARY)

            inverted_frame = cv2.bitwise_not(binary_frame)

            # Guarda la imagen mejorada
            cv2.imwrite("imagen_mejorada.png", inverted_frame)

            # Realiza OCR en la imagen mejorada
            text = pytesseract.image_to_string(inverted_frame, lang=idioma_ocr)

            # Aquí puedes hacer lo que necesites con el texto reconocido
            print("Texto reconocido:", text)

            if text != "":
                # Convierte el texto a discurso de voz
                tts = gTTS(text, lang=idioma_tts)

                # Genera un nombre de archivo único basado en el tiempo
                timestamp = int(time.time())
                audio_filename = f"texto_a_voz_{timestamp}.mp3"

                tts.save(audio_filename)

                # Reproduce el archivo de audio en tiempo real
                pygame.mixer.music.load(audio_filename)
                pygame.mixer.music.play()

        # Comprueba si se ha presionado 'Esc' para salir
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

capture_text_from_image()